<a href="https://www.kaggle.com/code/ranjitnayak/spaceship-data-to-alternate-dimension?scriptVersionId=101878752" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_full = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv', index_col='PassengerId')
X_test_full = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv', index_col='PassengerId')
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['Transported'], inplace=True)
y = X_full.Transported
X_full.drop(['Transported'], axis=1, inplace=True)
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()
X_train.head()


,HomePlanet,CryoSleep,Destination,VIP,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
PassengerId,,,,,,,,,,
4558_01,Europa,False,55 Cancri e,False,54.0,0.0,559.0,0.0,15238.0,2799.0
6326_01,Earth,False,TRAPPIST-1e,False,20.0,0.0,20.0,1.0,696.0,0.0
0503_02,Mars,False,TRAPPIST-1e,False,43.0,1821.0,0.0,47.0,29.0,0.0
4757_01,Earth,False,TRAPPIST-1e,False,24.0,185.0,0.0,476.0,1810.0,53.0
9046_01,Europa,True,55 Cancri e,False,25.0,0.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
numerical_transformer = SimpleImputer(strategy='constant')
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
model = XGBClassifier(n_estimators=1000, max_depth=10, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

[1 0 1 ... 1 1 0]
Your submission was successfully saved!
